In [ ]:
hostname = 'localhost'
username = 'postgres'
password = 'postgres'
port_id = 5432
database = 'pyspark_exercise_database'

In [ ]:
import psycopg2
import psycopg2.extras
import pandas as pd

### Every single scipt which was needed to create such a specified tables is written below. I am aware of the fact that some of the constraints are very obstructing and even some of them do not let insert appriopriate values into tables, but I tried to stick to demands. There is only one, small change, because I did not set unique constraint for column $CountryCode$ in table $city$, due to the fact that it were causing many errors when inserting values, despite dropping constraints with $DISABLE\:TRIGGER\:ALL$

In [ ]:
try:
    with psycopg2.connect (
        host = hostname,
        dbname = database,
        user = username,
        password = password,
        port = port_id) as conn:

        with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as cur:

            create_script0 = ''' CREATE TABLE IF NOT EXISTS pyspark_test.country (
                                    Code    char(3) PRIMARY KEY NOT NULL DEFAULT 'UNK' ,
                                    Name    varchar(255) NOT NULL DEFAULT '' ,
                                    Code2   char(2) NOT NULL UNIQUE DEFAULT '');
                            '''
                            
            alter_script0 = ''' ALTER TABLE pyspark_test.country ADD CHECK (LENGTH(pyspark_test.country.Code) > 2) '''

            create_script1 = ''' CREATE TABLE IF NOT EXISTS pyspark_test.city (
                                    ID    INT PRIMARY KEY NOT NULL,
                                    Name    varchar(255) NOT NULL DEFAULT '' ,
                                    CoutryCode   char(3) NOT NULL DEFAULT '',
                                    District    varchar(255) NOT NULL DEFAULT '',
                                    INFO    JSON DEFAULT NULL); '''

            alter_script1 = ''' ALTER TABLE pyspark_test.country ADD COLUMN 
                                    Capital int DEFAULT NULL;
            
                            '''

            alter_script2 = ''' ALTER TABLE pyspark_test.country
                                 ADD FOREIGN KEY (Capital)
                                 REFERENCES pyspark_test.city (ID);

                            '''

            enum_script0 = ''' CREATE TYPE IsOfficial AS ENUM ('T', 'F'); 

                            '''

            create_script2 = ''' CREATE TABLE IF NOT EXISTS pyspark_test.country_language (
                                    CountryCode     char(3) NOT NULL DEFAULT '',
                                    Language    char(30) NOT NULL DEFAULT '',
                                    IsOfficial    IsOfficial NOT NULL DEFAULT 'F',
                                    Percentage double precision NOT NULL DEFAULT 0,
                                    PRIMARY KEY (CountryCode, Language)
            );

            '''

            create_index_script0 = ''' CREATE INDEX code_index ON pyspark_test.country_language (CountryCode);
            
            '''

            create_sequence_script0 = ''' CREATE SEQUENCE seq_id
                                         MINVALUE 1
                                         START WITH 1
                                         INCREMENT BY 1
                                         CACHE 1;
            
            '''

            create_table_script_3 = ''' CREATE TABLE IF NOT EXISTS pyspark_test.country_info (
                                            doc     JSON,
                                            _id varchar(32) NOT NULL PRIMARY KEY DEFAULT nextval('seq_id')
            );
            
            '''

            all_scripts_to_execute = [create_script0, alter_script0, create_script1, alter_script1, alter_script2, enum_script0,
                                      create_script2, create_index_script0, create_sequence_script0, create_table_script_3]

            for script in all_scripts_to_execute:
                cur.execute(script)

except Exception as error:
    print(error)

finally:
    if conn is not None:
        conn.close()